# Crawler

#### Para criar os dois córpus foram utilizandos assuntos de tecnologia e mercado.
#### * Link para download dos corpora: https://drive.google.com/file/d/1IGUA3Wvqv_Dgihl5qqlr3pbVhHn1eC_1/view?usp=sharing

In [3]:
import urllib3
import re
from bs4 import BeautifulSoup
import os
import justext

In [4]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

http = urllib3.PoolManager(10, headers=user_agent)

In [5]:
class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path
        self.max_files = max_files
        self.seed_url = seed_url
        self.url_pattern = url_pattern
        self.visited_links = {}
        self.to_be_visited = []
        
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")), "w") as output_file:
            for paragraph in paragraphs:
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
        links = [link.get('href') for link in soup.find_all('a', attrs={'href': re.compile(self.url_pattern)})]

        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

#### O crawler utilizado, foi o mesmo disponibilizado pelo professor Erick

In [6]:
crawler_tecnologia = Crawler("data/corpora/tecnologia", 500, "https://www.diolinux.com.br/2016", "^https://www\.diolinux\.com\.br/\d+/\d+/\w+")
crawler_mercado = Crawler("data/corpora/mercado", 500, "https://www1.folha.uol.com.br/mercado/", "^https://www1\.folha\.uol\.com\.br/mercado/\w*")

#### Na hora de inicializar o objeto Crawler, foram feita algumas alterações no link de busca para juntar o total de 500 textos para cada córpus.
#### No blog do diolinux foi necessário ficar trocando o ano da url inicial da busca (2011 - 2019), a expressão regular nao precisou.
#### Para juntar os textos de mercado foi utilizado principalmente a Folha de São Paulo e o site do g1 na parte de economia https://g1.globo.com/economia/

In [7]:
crawler_tecnologia.crawl()

getting page https://www.diolinux.com.br/2016
getting page https://www.diolinux.com.br/2016/12/debian-9-atualizacoes-automaticas.html#comment-form
getting page https://www.diolinux.com.br/2016/12/android-things-google-anuncia-o-novo-sistema-IoT.html
getting page https://www.diolinux.com.br/2016/12/lancado-o-kdenlive-1612-linux.html
getting page https://www.diolinux.com.br/2016/12/amazon-lanca-servico-para-concorrer-com-netflix.html
getting page https://www.diolinux.com.br/2016/12/china-pede-o-codigo-fonte-do-windows-10.html
getting page https://www.diolinux.com.br/2016/12/microsoft-lanca-o-r-server-90-para-ubuntu.html


KeyboardInterrupt: 

In [8]:
crawler_mercado.crawl()

getting page https://www1.folha.uol.com.br/mercado/
getting page https://www1.folha.uol.com.br/mercado/2019/04/ministerio-de-damares-quis-afrouxar-proibicao-de-publicidade-para-crianca.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/avianca-cancela-mais-de-1300-voos-ate-2-de-maio-veja-lista.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/stf-amplia-em-r-16-bi-por-ano-subsidio-a-zona-franca-de-manaus.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/camara-deve-instalar-comissao-de-reforma-dos-militares-em-maio.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/pela-segunda-vez-ghosn-deixa-centro-de-detencao-apos-pagamento-de-fianca.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/parlamentar-esta-preocupado-com-custo-de-oportunidade-de-nao-aprovar-reforma-diz-eurasia.shtml
getting page https://www1.folha.uol.com.br/mercado/2019/04/camara-instala-comissao-especial-e-inicia-prazo-de-tramitacao-da-previdencia.s

KeyboardInterrupt: 

# Estatísticas

In [3]:
import os
from normalib.lexical import Normalization
from normalib.statistics import Statistics

In [4]:
corpora_path = 'Crawler/data/corpora/'
files_tecnologia = os.listdir('{}/tecnologia/'.format(corpora_path))
files_tecnologia = ['{}/tecnologia/{}'.format(corpora_path,f) for f in files_tecnologia if f != '.DS_Store']
files_mercado = os.listdir('{}/mercado/'.format(corpora_path))
files_mercado = ['{}/mercado/{}'.format(corpora_path,f) for f in files_mercado if f != '.DS_Store']

In [5]:
corpus_lines_tecnologia = []
for file in files_tecnologia:
    with open(file, "r") as text_file:
        lines = text_file.readlines()
        corpus_lines_tecnologia.extend(lines)

In [6]:
corpus_lines_mercado = []
for file in files_mercado:
    with open(file, "r") as text_file:
        lines = text_file.readlines()
        corpus_lines_mercado.extend(lines)

#### Depois de ler todos os textos de tecnologia e mercado e acumulá-los em uma lista, agora vamos utilizar a biblioteca criada para gerar algumas estatísticas.

In [7]:
n = Normalization()
s = Statistics()

In [8]:
all_sentences_tecnologia = n.normalization_pipeline(' '.join(corpus_lines_tecnologia), to_lower_case=True,
                                                    remove_accents=False,
                                                    remove_punctuation=True, remove_stopwords=True, lemmatize=False,
                                                    stemmize=False, tokenize_sentences=False, tokenize_words=False)

all_sentences_mercado = n.normalization_pipeline(' '.join(corpus_lines_mercado), to_lower_case=True,
                                                    remove_accents=False,
                                                    remove_punctuation=True, remove_stopwords=True, lemmatize=False,
                                                    stemmize=False, tokenize_sentences=False, tokenize_words=False)

#### Aqui no pipeline vamos utilizar somente to_lower_case, remove_stopwords e remove_punctuation, para melhorar na busca das 20 palavras mais e menos utilizadas.
#### * Logo abaixo serão listados respectivamente, palavras mais e menos frequentes de tecnologia e depois de mercado

In [9]:
', '.join(s.frequently_words(all_sentences_tecnologia))

'é, linux, ubuntu, pode, sistema, ser, versão, ainda, fazer, feira, pessoas, windows, vai, sobre, instalar, dezembro, agora, ano, forma, alguns'

In [11]:
', '.join(s.frequently_words(all_sentences_tecnologia, more=False))

'observadas, exploradas, danger, programaacadêmicosuse, massividade, identação, permanecerem, competentes, defesas, atom, mobil, estimados, dormir, fish, obrigatória, autorna, checagem, desgraçado, 2018photofilmstrip, constando'

In [13]:
', '.join(s.frequently_words(all_sentences_mercado))

'é, r, ser, paulo, governo, anos, sobre, meio, todos, folha, ano, qualquer, direitos, reforma, desta, presidente, s, diz, comunicação, previdência'

In [14]:
', '.join(s.frequently_words(all_sentences_mercado, more=False))

'mobilize, convincentemente, triclosan, totalizou, discriminados, repará, permanecerem, gozações, suspensos, emoções, medieval, receptivos, 1recém, intrínseco, pensadores, barbeiro, seda, equipethe, encolhendo, pessoalcopiar'

####  * Logo abaixo serão listados respectivamente, o tamanho medio das palavras de tecnologia e mercado

In [10]:
print('Tamanho médio palavras corpus tecnologia: ' + str(s.average_size_words(all_sentences_tecnologia)))

Tamanho médio palavras corpus tecnologia: 6.397491312823121


In [11]:
print('Tamanho médio palavras corpus mercado: ' + str(s.average_size_words(all_sentences_mercado)))

Tamanho médio palavras corpus mercado: 6.741267932713277


#### * Logo abaixo serão listados respectivamente, o tamanho médio das setenças em número de palavras de tecnologia e depois mercado. Para isso será necessário processar os dados novamente, pois vamos precisar utilizar pontuação, visto que nas estatísticas calculadas acima os resultados ficam melhores sem pontuação

In [12]:
sentences_tecnologia_punctuation = n.normalization_pipeline(' '.join(corpus_lines_tecnologia), to_lower_case=True,
                                                    remove_accents=False,
                                                    remove_punctuation=False, remove_stopwords=True, lemmatize=False,
                                                    stemmize=False, tokenize_sentences=False, tokenize_words=False)

sentences_mercado_punctuation = n.normalization_pipeline(' '.join(corpus_lines_mercado), to_lower_case=True,
                                                    remove_accents=False,
                                                    remove_punctuation=False, remove_stopwords=True, lemmatize=False,
                                                    stemmize=False, tokenize_sentences=False, tokenize_words=False)

In [13]:
print('Tamanho médio sentenças corpus tecnologia: ' + str(s.average_size_sentences(sentences_tecnologia_punctuation)))

Tamanho médio sentenças corpus tecnologia: 40.00988189925283


In [14]:
print('Tamanho médio sentenças corpus mercado: ' + str(s.average_size_sentences(sentences_mercado_punctuation)))

Tamanho médio sentenças corpus mercado: 27.181781539701735


#### Agora vamos calcular a quantidade de vezes que uma palavra qualquer aparece em nosso corpus de tecnologia e mercado, para isso vamos usar a variavel que guarda os dados sem pontuação.
#### * Primeiro vamos calcular a palavra linux no corpus tecnologia

In [15]:
print('Corpus tecnologia: ' + str(s.word_appear(all_sentences_tecnologia, 'linux')))

Corpus tecnologia: 1671


#### * Agora vamos calcular a palavra dinheiro no corpus mercado

In [16]:
print('Corpus mercado: ' + str(s.word_appear(all_sentences_tecnologia, 'dinheiro')))

Corpus mercado: 55


#### * Agora vamos mostrar quantas palavras apareceram somente 1 vez no corpus de tecnologia e mercado

In [17]:
print('Corpus tecnologia: ' + str(s.word_once_length(all_sentences_tecnologia)))

Corpus tecnologia: 5516
